In [347]:
import requests
import json
import pandas as pd
from fpdf import FPDF

In [348]:
def extract():
    api_key = ...
    player_stats = requests.get(
        'https://api.sportsdata.io/v3/nba/stats/json/PlayerSeasonStatsByTeam/2022/LAL?key=38ef11def74d4f13a039690fa9cde9b4').json()
    return player_stats


def write_to_json_file(data):
    with open('nba_stats.json', 'w') as f:
        json.dump(data, f)


def transform(data):
    df = pd.DataFrame(data)
    interesting_columns = ['Season', 'Team', 'Position', 'Games', 'FantasyPoints', 'Minutes', 'TwoPointersMade', 'TwoPointersAttempted', 'TwoPointersPercentage', 'ThreePointersMade', 'ThreePointersAttempted', 'ThreePointersPercentage', 'FreeThrowsMade', 'FreeThrowsAttempted', 'FreeThrowsPercentage', 'OffensiveRebounds', 'DefensiveRebounds', 'Rebounds', 'Assists', 'Steals', 'BlockedShots', 'Turnovers', 'PersonalFouls', 'Points', 'PlayerEfficiencyRating']
    # df = df.pivot_table(index=['Name'], values=interesting_columns, sort=False)
    df = df.set_index('Name')
    df = df[interesting_columns]
    column_names = {
        'Season': 'Season',
        'Team': 'Team',
        'Position': 'Pos',
        'Games': 'G',
        'FantasyPoints': 'FP',
        'Minutes': 'Min',
        'TwoPointersMade': '2PM',
        'TwoPointersAttempted': '2PA',
        'TwoPointersPercentage': '2P%',
        'ThreePointersMade': '3PM',
        'ThreePointersAttempted': '3PA',
        'ThreePointersPercentage': '3P%',
        'FreeThrowsMade': 'FTM',
        'FreeThrowsAttempted': 'FTA',
        'FreeThrowsPercentage': 'FT%',
        'OffensiveRebounds': 'OffReb',
        'DefensiveRebounds': 'DefReb',
        'Rebounds': 'Reb',
        'Assists': 'Ast',
        'Steals': 'Stl',
        'BlockedShots': 'Blk',
        'Turnovers': 'TO',
        'PersonalFouls': 'PF',
        'Points': 'Pts',
        'PlayerEfficiencyRating': 'Rtg'
    }
    df.index.name = None
    for _ in range(len(interesting_columns)):
        df.rename(columns=column_names, inplace=True)
    # Change column names
    df.sort_values(by=['FP'], ascending=False, inplace=True)
    # with open('stats_nba.csv', 'w') as f:
    #     df.to_csv(f)
    return df


def create_pdf(df):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(40, 10, 'Los Angeles Lakers')
    pdf.ln()
    pdf.ln()
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(40, 10, 'Season 2022')
    pdf.ln()
    pdf.set_font('Arial', 'B', 10)
    pdf.cell(40, 10, 'Player stats')
    pdf.ln()
    lengths = {
        'Season': 9,
        'Team': 7,
        'Pos': 6,
        'G': 4,
        'FP': 8,
        'Min': 7,
        '2PM': 7,
        '2PA': 7,
        '2P%': 6,
        '3PM': 7,
        '3PA': 7,
        '3P%': 6,
        'FTM': 7,
        'FTA': 7,
        'FT%': 6,
        'OffReb': 9,
        'DefReb': 9,
        'Reb': 7,
        'Ast': 7,
        'Stl': 6,
        'Blk': 6,
        'TO': 7,
        'PF': 7,
        'Pts': 7,
        'Rtg': 6
    }
    pdf.set_font('Arial', 'B', 5)
    pdf.cell(18, 8, 'Player', border=1)
    for key, value in lengths.items():
        pdf.cell(value, 8, key, border=1)
    pdf.ln()
    pdf.set_font('Arial', '', 5)
    for index, row in df.iterrows():
        pdf.cell(18, 4, index, border=1)
        for key, value in lengths.items():
            pdf.cell(value, 4, str(row[key]), border=1)
        pdf.ln()
    pdf.multi_cell(0, 2, '\n* Pos = Position, G = Games, FP = Fantasy Points, Min = Minutes, 2PM = Two Pointers Made, 2PA = Two Pointers Attempted, 2P% = Two Pointers Percentage, 3PM = Three Pointers Made, 3PA = Three Pointers Attempted, 3P% = Three Pointers Percentage, FTM = Free Throws Made, FTA = Free Throws Attempted, FT% = Free Throws Percentage, OffReb = Offensive Rebounds, DefReb = Defensive Rebounds, Reb = Rebounds, Ast = Assists, Stl = Steals, Blk = Blocks, TO = Turnovers, PF = Personal Fouls, Pts = Points, Rtg = Player Efficiency Rating')
    pdf.output('nba_stats.pdf', 'F')
    

In [349]:
stats = extract()


In [350]:
write_to_json_file(stats)


In [351]:
df = transform(stats)
display(df)


,Season,Team,Pos,G,FP,Min,2PM,2PA,2P%,3PM,...,OffReb,DefReb,Reb,Ast,Stl,Blk,TO,PF,Pts,Rtg
Russell Westbrook,2022,LAL,PG,78,2859.9,2680,469.5,969.0,48.5,79.1,...,110.1,470.5,580.6,550.6,75.1,20.0,295.3,235.2,1442.4,18.4
LeBron James,2022,LAL,SF,56,2840.1,2086,479.5,773.8,62.1,161.2,...,63.1,396.4,459.5,349.3,73.1,59.1,196.2,121.1,1696.7,31.1
Anthony Davis,2022,LAL,PF,40,1780.6,1403,357.4,625.6,57.2,13.0,...,106.1,288.3,394.4,122.1,49.0,90.1,82.1,97.1,927.9,28.0
Malik Monk,2022,LAL,SG,76,1741.9,2140,218.2,384.4,56.9,173.2,...,37.0,219.2,256.3,218.2,61.1,29.0,122.1,138.1,1049.0,17.3
Carmelo Anthony,2022,LAL,PF,69,1507.1,1795,170.2,326.3,52.2,149.1,...,62.1,226.2,288.3,68.1,47.0,52.1,59.1,166.2,919.9,17.6
Talen Horton-Tucker,2022,LAL,SG,60,1162.3,1512,175.2,356.4,49.2,49.0,...,37.0,156.2,193.2,159.2,59.1,29.0,82.1,148.1,597.6,13.4
Dwight Howard,2022,LAL,C,60,945.6,974,126.1,204.2,61.9,8.0,...,118.1,238.2,356.4,35.0,34.0,36.0,47.0,115.1,372.4,20.7
Austin Reaves,2022,LAL,SG,61,899.5,1418,93.1,151.2,61.7,53.1,...,45.0,148.1,193.2,110.1,29.0,20.0,40.0,87.1,444.4,13.6
Avery Bradley,2022,LAL,SG,62,714.4,1406,66.1,140.1,47.1,82.1,...,31.0,105.1,136.1,47.0,53.1,9.0,38.0,118.1,394.4,9.5
Stanley Johnson,2022,LAL,SF,48,705.7,1095,84.1,146.1,57.6,33.0,...,34.0,121.1,155.2,80.1,43.0,14.0,35.0,109.1,320.3,12.4


In [352]:
create_pdf(df)